# Supervisor Multi AI Agent Architecture

In [ ]:
from typing import Annotated, TypedDict, List, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
# from typing_extensions import TypedDict

from langgraph.graph import END, START,StateGraph, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq

import random
from datetime import datetime

import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Defining supervisor class
class SupervisorState(MessagesState):
    """State for the supervisor multi-agent system"""
    next_agent: str = ""
    research_data: str = ""
    analysis: str = ""
    final_report: str = ""
    task_complete: bool = False
    current_task: str = ""

# creating supervisor chain
def create_supervisor_chain():
    """creates a supervisor decision chain"""
    supervisor_prompt = ChatPromptTemplate.from_messages([
        ("system", """"You are a supervisor managing a team of agents:
            
            1. Researcher - Gathers information and data
            2. Analyst - Analyses the data and provides insights
            3. Writer - Creates reports and summaries

            Based on the current state and conversation , decide which agent should work next.
            If the task is completed, respond with 'Done'.

            Current state:
            - Has research data : {has_research}
            - Has analysis : {has_analysis}
            - Has report : {has_report}

            Respond with ONLY the agent name (researcher/analyst/writer) or 'Done'.
        """ ),
        ("human","{task}")
    ])

    return supervisor_prompt | llm